In [1]:
import torch
import torch2trt
import matplotlib.pyplot as plt
import cv2
import segmentation_models_pytorch as smp

In [3]:
model = smp.Unet(
    encoder_name="mobilenet_v2",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pretrained weights for encoder initialization
    classes=1,
    activation="sigmoid",          # model output channels (number of classes in your dataset)
)

In [4]:
model = model.cuda().eval()

In [5]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
encoder.features.0.0.weight 	 torch.Size([32, 3, 3, 3])
encoder.features.0.1.weight 	 torch.Size([32])
encoder.features.0.1.bias 	 torch.Size([32])
encoder.features.0.1.running_mean 	 torch.Size([32])
encoder.features.0.1.running_var 	 torch.Size([32])
encoder.features.0.1.num_batches_tracked 	 torch.Size([])
encoder.features.1.conv.0.0.weight 	 torch.Size([32, 1, 3, 3])
encoder.features.1.conv.0.1.weight 	 torch.Size([32])
encoder.features.1.conv.0.1.bias 	 torch.Size([32])
encoder.features.1.conv.0.1.running_mean 	 torch.Size([32])
encoder.features.1.conv.0.1.running_var 	 torch.Size([32])
encoder.features.1.conv.0.1.num_batches_tracked 	 torch.Size([])
encoder.features.1.conv.1.weight 	 torch.Size([16, 32, 1, 1])
encoder.features.1.conv.2.weight 	 torch.Size([16])
encoder.features.1.conv.2.bias 	 torch.Size([16])
encoder.features.1.conv.2.running_mean 	 torch.Size([16])
encoder.features.1.conv.2.running_var 	 torch.Size([16])
encoder.features.1.conv.2.num_batches

NameError: name 'optimizer' is not defined

In [6]:
# Load the jetracer trained model 
model.load_state_dict(torch.load('/models/run10/state_dict.pt'))

RuntimeError: Error(s) in loading state_dict for Unet:
	Missing key(s) in state_dict: "encoder.features.0.0.weight", "encoder.features.0.1.weight", "encoder.features.0.1.bias", "encoder.features.0.1.running_mean", "encoder.features.0.1.running_var", "encoder.features.1.conv.0.0.weight", "encoder.features.1.conv.0.1.weight", "encoder.features.1.conv.0.1.bias", "encoder.features.1.conv.0.1.running_mean", "encoder.features.1.conv.0.1.running_var", "encoder.features.1.conv.1.weight", "encoder.features.1.conv.2.weight", "encoder.features.1.conv.2.bias", "encoder.features.1.conv.2.running_mean", "encoder.features.1.conv.2.running_var", "encoder.features.2.conv.0.0.weight", "encoder.features.2.conv.0.1.weight", "encoder.features.2.conv.0.1.bias", "encoder.features.2.conv.0.1.running_mean", "encoder.features.2.conv.0.1.running_var", "encoder.features.2.conv.1.0.weight", "encoder.features.2.conv.1.1.weight", "encoder.features.2.conv.1.1.bias", "encoder.features.2.conv.1.1.running_mean", "encode

In [15]:
class ModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(ModelWrapper, self).__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)['out']

In [18]:
model_w = ModelWrapper(model)

In [19]:
data = torch.ones((1,3,320,640)).cuda()

In [20]:
model_trt = torch2trt.torch2trt(model_w, [data], fp16_mode=True)

TypeError: new(): invalid data type 'str'

# Live demo

In [ ]:
ino = 289
# Read  a sample image and mask from the data-set
img = cv2.imread(f'/models/train_data/Images/{ino:03d}_cam-image1_.jpg').transpose(2,0,1).reshape(1,3,300,650)
mask = cv2.imread(f'/models/train_data/Masks/{ino:03d}_cam-image1_mask.png')
input = torch.from_numpy(img).type(torch.cuda.FloatTensor)/255

In [ ]:
with torch.no_grad():
    output = model(input)

In [ ]:
# Plot histogram of the prediction to find a suitable threshold. From the histogram a 0.1 looks like a good choice.
plt.hist(output['out'].data.cpu().numpy().flatten())

In [ ]:
# Plot the input image, ground truth and the predicted output
plt.figure(figsize=(10,10));
plt.subplot(131);
plt.imshow(img[0,...].transpose(1,2,0));
plt.title('Image')
plt.axis('off');
plt.subplot(132);
plt.imshow(mask);
plt.title('Ground Truth')
plt.axis('off');
plt.subplot(133);
plt.imshow(output['out'].cpu().detach().numpy()[0][0]>0.4);
plt.title('Segmentation Output')
plt.axis('off');

In [ ]:
output_trt = model_trt(input)

In [ ]:
# Plot histogram of the prediction to find a suitable threshold. From the histogram a 0.1 looks like a good choice.
plt.hist(output_trt['out'].data.cpu().numpy().flatten())

In [ ]:
# Plot the input image, ground truth and the predicted output
plt.figure(figsize=(10,10));
plt.subplot(131);
plt.imshow(img[0,...].transpose(1,2,0));
plt.title('Image')
plt.axis('off');
plt.subplot(132);
plt.imshow(mask);
plt.title('Ground Truth')
plt.axis('off');
plt.subplot(133);
plt.imshow(output_trt['out'].cpu().detach().numpy()[0][0]>0.4);
plt.title('Segmentation Output')
plt.axis('off');